## Importing required packages

In [115]:
import re
import pandas as pd 
from urlparse import urlsplit

## Reading all files

In [116]:
main_file=sc.textFile("hdfs://localhost:8020/pkdd2005/logs/FlumeData*")

In [117]:
main_file.first() ## printing the test

u'14;1074585600;147.33.10.112;89ccfad2c4bbc02c91ed66055a235fca;/ls/index.php?&id=62&view=1,2,3,4,6,9&sort=,13,4&pozice=40;http://www.shop4.cz/ls/index.php?&id=62&view=1,2,3,4,6,9&sort=,13,4&pozice=20'

In [118]:
main_file.count()

3617171

##Creating transformation functions

In [119]:
def transformsnumer(inputStr):
    attList=inputStr.split(";")
    
    x =attList[4]
    y =attList[5]
    
    if len(x)>2:
        Visited1 = visited_Transform(x)
        ID1=ID_transform(x)
    else:
        Visited1 =''
        ID1=''
    if len(y)>2:
        Visited2= visited_Transform(y)
        ID2=ID_transform(y)
        
    else:
        Visited2=''
        ID2=''
        
    values= ([Visited1,ID1,Visited2,ID2])
    
    return values   

In [120]:
def find_nth_character(str1, substr, n):
    pos = -1
    for x in xrange(n):
        pos = str1.find(substr, pos+1)
        if pos == -1:
            return None
    return pos

In [121]:
def visited_Transform(intstr):
    parsed = urlsplit(intstr)
    try:
        if len(parsed.path)==0 or parsed.path =='/' or parsed.path[0:5]=='w-ans':
            values=''
        
        else:
        
            if parsed.path[3]=='/':
                values=parsed.path[0:4]
            else:
            
                c = '/'
                x = parsed.path
                a=find_nth_character(x,c,2)
                if a > 0:
                    values=parsed.path[0:a+1]
                else:
                    values=parsed.path
    except:
        values=''
    return values

In [122]:
def ID_transform(intstr):
    parsed = urlsplit(intstr)
    x= parsed.query
    y=parsed.path
    try:
        if len(x) == 0:
            values=''
        else:
            if y[0:4]=='/dt/':
                c= '=' 
                a=find_nth_character(x,c,1)
                values=x[a+1:]
            elif y[0:4]=='/ls/':
                if parsed.query[0]=='&':
                    c='&'
                    d='id='
                    a=find_nth_character(x,d,1)
                    b=find_nth_character(x,c,2)
                    values=parsed.query[a+3:b]
#                 except:
#                     values=''

                else:
                    if len(parsed.query)<=6:
                        #c=len()
                        d='id='
                        a=find_nth_character(x,d,1)
                        #b=find_nth_character(x,c,2)
                        values=parsed.query[a+3:]
                    else:
                        c='&'
                        d='id='
                        a=find_nth_character(x,d,1)
                        b=find_nth_character(x,c,2)
                        values=parsed.query[a+3:b]
            else:
                parsed.path[0:4] =='/ct/'
                values=parsed.query[2:6]
    except:
        values =''
    return  values

## Transforming one RDD to another

In [123]:
main_files_conv = main_file.map(transformsnumer)

In [124]:
main_files_conv.take(10)

[[u'/ls/', u'62', u'/ls/', u'62'],
 [u'/ct/', u'158', '', ''],
 [u'/klient/', '', u'/klient/', ''],
 [u'/dt/', u'9354', '', ''],
 ['', '', '', ''],
 [u'/ls/', u'67', u'/ls/', u'67'],
 [u'/dt/', u'3475', u'/ls/', u'62'],
 [u'/dt/', u'10687', u'/ls/', u'18'],
 [u'/ls/', u'62', u'/ls/', u'62'],
 [u'/ls/', u'3', u'/ct/', u'141']]

In [125]:
Entire_Files_DF= main_files_conv.toDF(["Visited","ID","Referrer","ID_Referrer"]) ## transfering rdd to sqlDF

In [126]:
Entire_Files_DF.show(10)

Visited  ID    Referrer ID_Referrer
/ls/     62    /ls/     62         
/ct/     158                       
/klient/       /klient/            
/dt/     9354                      
                                   
/ls/     67    /ls/     67         
/dt/     3475  /ls/     62         
/dt/     10687 /ls/     18         
/ls/     62    /ls/     62         
/ls/     3     /ct/     141        


## Querying pages where visited = '/ls/-product listing' and referrer = '/ct/-product category '

In [127]:
Final_Analysis1=Entire_Files_DF.where(Entire_Files_DF.Visited=='/ls/') ## subset the dataframe based on first condt

In [128]:
Final_Analysis1.count()

1363187L

In [129]:
Final_Analysis=Final_Analysis1.where(Final_Analysis1.Referrer=='/ct/') ## subset the dataframe based on second condt

In [130]:
Final_Analysis.count()

179512L

In [131]:
Final_Analysis.show(10)

Visited ID  Referrer ID_Referrer
/ls/    3   /ct/     141        
/ls/    38  /ct/     158        
/ls/    106 /ct/     505        
/ls/    3   /ct/     141        
/ls/    33  /ct/     155        
/ls/    51  /ct/     163        
/ls/    34  /ct/     155        
/ls/    70  /ct/     171        
/ls/    74  /ct/     172        
/ls/    134 /ct/     506        


In [132]:
Last_trans= Final_Analysis.where(Final_Analysis.ID_Referrer>0) ## subset the dataframe removing blank lines on visit_ID
#Last_tans.count()

In [133]:
Final_Df=Last_trans.where(Last_trans.ID>0)## subset the dataframe removing blank lines on referrer_ID
Final_Df.count()

174258L

In [46]:
Final_Df.show(10)

Visited ID  Referrer ID_Referrer
/ls/    3   /ct/     141        
/ls/    38  /ct/     158        
/ls/    106 /ct/     505        
/ls/    3   /ct/     141        
/ls/    33  /ct/     155        
/ls/    51  /ct/     163        
/ls/    34  /ct/     155        
/ls/    70  /ct/     171        
/ls/    74  /ct/     172        
/ls/    134 /ct/     506        


##Converting to Pandas for visualizing in Neo4j (Graph DB)

In [134]:
FinalPD= Final_Df.toPandas()

In [135]:
Final_output=FinalPD.groupby(["ID", "ID_Referrer"]).size().reset_index(name="Frequency")

In [136]:
Final_output.head(10)

,ID,ID_Referrer,Frequency
0,1,140,873
1,1,141,1
2,10,145,2466
3,10,160,186
4,10,173,4
5,10,174,11
6,10,465,10
7,100,388,432
8,101,401,1650
9,102,401,635


In [157]:
Final_list=Final_output.values.tolist()

##Importing dimension tables

In [137]:
category=sc.textFile("hdfs://localhost:8020/user/training/assignment/category/*")
category.first()

u'140\tKlasick\xe9 fotoapar\xe1ty\tFilm cameras'

In [138]:
def transformers(intstr):
    attLst =intstr.split("\t")
    values=([attLst[0],attLst[2]])
    return values

In [139]:
cat=category.map(transformers)
cat.first()

[u'140', u'Film cameras']

In [140]:
category_df=cat.toDF(["id","category_name"])
category_df.show(10)

id  category_name       
140 Film cameras        
141 Digital cameras     
142 Zoom lenses         
143 Standard lenses     
144 Accessories         
145 Data recording media
146 HiFi systems        
147 HiFi components     
148 Earphones           
149 Cassette radios     


In [141]:
category_Pdf=category_df.toPandas()

In [218]:
category_list=category_Pdf.values.tolist()

In [143]:
product=sc.textFile("hdfs://localhost:8020/user/training/assignment/list/*")
product.first()

u'1\tKlasick\xe9 fotoapar\xe1ty\tFilm cameras'

In [144]:
prod=product.map(transformers)

In [145]:
product_df=prod.toDF(["id","product_name"])
product_df.show(10)

id product_name        
1  Film cameras        
2  Film SLR cameras    
3  Digital cameras     
4  Digital SLR cameras 
5  Lenses - manual zoom
6  Lenses - auto zoom  
7  Lenses - manual f...
8  Lenses - auto fixed 
9  Flash lights        
10 Memories            


In [146]:
product_Pdf=product_df.toPandas()

In [214]:
product_Pdf.head(10)

,id,product_name
0,1,Film cameras
1,2,Film SLR cameras
2,3,Digital cameras
3,4,Digital SLR cameras
4,5,Lenses - manual zoom
5,6,Lenses - auto zoom
6,7,Lenses - manual fixed
7,8,Lenses - auto fixed
8,9,Flash lights
9,10,Memories


In [217]:
product_list=product_Pdf.values.tolist()

##Importing required packages from neo4j

In [163]:
from py2neo import Graph, Path
from py2neo import Node,Relationship
from py2neo import authenticate

In [164]:
authenticate("localhost:7474","training","training")

In [165]:
graph=Graph("http://localhost:7474/db/data/")

###Creating nodes and relationships; with properties

In [190]:
ctx = graph.cypher.begin()

In [191]:
for id,category_name in category_list:
    ctx.append("create (c:category {name:{category_name}, id:{id}}) return c",id=id,category_name=category_name)
    #tx.append("CREATE (person:Person {name:{name}}) RETURN person", name=name)

In [192]:
category = [result.one for result in ctx.commit()]

In [193]:
ptx = graph.cypher.begin()

In [194]:
for id,product_name in product_list:
    ptx.append("create (p:product {name:{product_name}, id:{id}}) return p",id=id,product_name=product_name)
    #tx.append("CREATE (person:Person {name:{name}}) RETURN person", name=name)

In [195]:
product=[result.one for result in ptx.commit()]

In [196]:
for ID,ID2,Frequency in Final_list:
    graph.cypher.execute("match(p:product {id:{id1}})\
                         match(c:category {id:{id2}})\
                         create ((c)-[:refered{frequency:{frequency}}]->(p))",id1=ID,id2=ID2,frequency=Frequency)

##Future scope

#### > Recommendation based on User patterns

#### > Include Brands in the analysis

#### > Identify Fraud based on the entire session